In [1]:
%autosave 1

Autosaving every 1 seconds


- KNN
    - model capacity

- Linear classifiers
    - as template matching
    - loss
        - 0-1 loss
    - softmax
    - cross-entropy loss
    - gradient descent

- SGD
    - SGD with minibatches
    - online learning
    - problems of SGD
- Momentum
- Nesterov momentum
- AdaGrad
- RMSProp
- ADAM

- representation learning
- activation function
    - its purpose
- Neural Network

- convolution
    - properties
    - formula of learnable parameters
    - formula of MB to store in memory
    - formula of flops
- input image -- conv2D -- output shapes relationships (also for multiple layers)
- formula for H_out and W_out
    - after a conv2D
    - after a conv2D with padding
    - after a conv2D with padding and stride
- pooling layers
    - pro and cons
    - formula of learnable parameters
    - formula of MB to store in memory
    - formula of flops
- Batch Normalization
    - internal covariance shift
    - training time
    - test time
    - pro and cons

- AlexNet
    - trends
    - general performance
- ZFNet / Clarify
    - (visualization of kernels and activations)
    - general performance
- VGG
    - stages
    - its three main choices
    - (no stemming)
    - general performance
- Inception v1
    - (stem - inception modules - GlobalAVGPooling)
    - naive inception module
    - 1x1 conv
    - real inception module
    - GlobalAVGPooling
    - Inception v3
    - general performance
- Residual Networks
    - residual block
        - how is formed
            - skip connections
            - two 3x3 conv
        - halve spatial resolution, doubles channels
        - uses stem and GlobalAVGPooling
    - skip connection dimension problem
        - problem of 3/4 discarded pixel of the 1x1 conv with s=2
            - solved by adding a 2x2 AvgPool layer before the 1x1 conv with s=2
    - bottleneck residual block
    - effects of residual learning
- ResNeXt
    - idea
    - argue the growing complexity of 3x3 convs
        - compute flops and solve for *d*
    - why ResNeXt idea is a good one? 
    - grouped convs
- SENet
    - capture global context
    - squeeze part: GlobalAVGPooling
    - excitation part: outputs weight to reweight the channels
        - *r* reductionf factor
        - relative importance of channels
- Depthwise Separable conv
    - extreme grouped conv with #groups==C
- Inverted residual block
    - why Bottleneck residual block are not ok
    - expansion - process - compression
    - *t* expansion rate
    - MobileNet-v2
        - stack of inverted residual block
- Wide ResNet
    - ResNet with channels multiplied by a facto *k*
- EfficientNet
    - "what is the optimal way to scale up a model?"
    - single dimension scaling
        - all three saturates at 80%
    - compound scaling: scaling W, D and R in an optimal way to improve the most we can
        - compound scaling $\phi$
        - formulation
    - NAS (Neaural Architecture Search)

- model capacity
    - factors the infuences it
- regularization
    - increase bias paying training error
- parameter norm penalties
    - optimize another term of the loss which is conflicting that say:
        - "we want our params to have small values"
        - Lambda hyperparameter
    - weigh decay
- early stopping
- label smoothing
    - problem of one hot encoding of labels
        - making model overly confident: overfitting
    - better alternative: smooth the labels
        - this accounts for mislabeled examples
    - how to apply labels smoothing
    - KLDiv loss
- dropout
    - in forward pass we use a subset of the network
        - hyperparameter *p* zeroing activation
    - why is this a good idea?
        - prevents feature detectors to co-adapt
            - face detector example
    - test time preds are stochastic
        - value at test time
        - expected value at training time with p=0.5
            - example
            - inverted drop out
- data augmentation
    - multi-scale training
    - multi-scale testing
        - domain shift problem
        - second alternative to multi-scale testing
- color augmentation (jittering)
- cutout
- Mixup
    - linear combination of two images according to a weight lambda
        - lambda picked from a Beta distribution
    - why is a good idea?
        - contraints what the network does between classes
    - testing
        - unmodified input

- learning rate schedule
    - step decay 
    - cosine decay
    - linear
    - warm-up
        - to use when our trainig loss is flattened for a long time
    - one cycle
        - update the learning rate after each interation, not epoch
        - vary momentum
- random hyper-parameter search
- recipe to train a NN
    - test time: ensemble
    - snaphot ensambling
        - uses cyclic cosine decay
        - majory voting at test time of M models
    - Polyak average
        - eponential moving average of parameters
    - Stochastic Weight Averaging
        - uses cyclic learning rates
        - real running average only when the learning rate is decreased

- Transfer Learning
    - First way
        - freeze backbone and train just the last layer
    - Second way
        - train everything
            - discrepancy between last layer and backbone
            - keep frozen backbne for few epochs untul last layer goes into a good landscape
            - unfreeze backbone and train with e-4 lr if it was e-3
            - Progressive LRs: first layers are ok so we freeze them
                - a growing lr when we go deep into the net to be more task specific

- Detecting multiple objects
    - problem 1: background
    - problem 2: too many possible windows
    - solution: region proposal
        - apply with Selective Search to come up with regions that are likely to contain obj
- R-CNN
    - run Selective Search to come up with for example 2000 proposals
    - for each of this proposal:
        - warp it adding 16 pixels of context
        - pass through the Net
        - get class and BB correction
    - problem: really slow
- Fast R-CNN
    - still run Selective Search to come up with for example 2000 proposals
    - run full image up to a certain conv layer (like conv5) only once
    - project the proposal into the resulting activation
    - use RolPool layer to crop the projections and resize to the right shape
    - advantage: the 2000 proposals pass only to a small part and non-expensive of the net
        - which are the FC layers at the end

QUESTIONS:
- dilated convs
    - why are useful, its advantages
- what algo do we use to train NN
    - what are the hyperparameters that influences the training (learning rate, batch size)
    - effect of learning rate
    - effect of smaller and bigger batch size
- regularization
    - approaches we use to improve it
        - labels smoothing
            - why is useful, how it works
            - softmax and CE formula
- metric learning
    - why we need triplette loss, what it is and what it improves
    - contrastive loss vs triplette loss
    - triplette loss formula
    - do we take all possible triplettes or just a subset?
        - semi hard negative mining
            - how we define an example being semi hard negative?